## Day 29 Lecture 2 Assignment

In this assignment, we will learn about entropy and information gain in the ID3 algorithm.

In [283]:
import numpy as np
import pandas as pd

from math import log2

In [10]:
tennis = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/tennis_decision.csv')

In [11]:
tennis

,Day,Outlook,Temp.,Humidity,Wind,Decision
0,1,Sunny,Hot,High,Weak,No
1,2,Sunny,Hot,High,Strong,No
2,3,Overcast,Hot,High,Weak,Yes
3,4,Rain,Mild,High,Weak,Yes
4,5,Rain,Cool,Normal,Weak,Yes
5,6,Rain,Cool,Normal,Strong,No
6,7,Overcast,Cool,Normal,Strong,Yes
7,8,Sunny,Mild,High,Weak,No
8,9,Sunny,Cool,Normal,Weak,Yes
9,10,Rain,Mild,Normal,Weak,Yes


In [339]:
outlook = tennis.groupby('Decision').count()

In [353]:
outlook = pd.crosstab(index=tennis.Decision, columns=tennis.Outlook)
temp = pd.crosstab(index=tennis.Decision, columns=tennis['Temp.'])
hum = pd.crosstab(index=tennis.Decision, columns=tennis.Humidity)
wind = pd.crosstab(index=tennis.Decision, columns=tennis.Wind)

In [354]:
getval(outlook)

no_entropy: 0.9709505944546686
yes_entropy: 0.9709505944546686
weighted avg. entropy: 0.9709505944546686


In [84]:
vars = [outlook, temp, hum, wind]

In [369]:
def getval(df):
    total_no = df.iloc[0].sum()
    total_yes = df.iloc[1].sum()
    total_val = total_no + total_yes

    no_ratio = []
    yes_ratio = []

    for col in df:
        col_total = df.iloc[0][col] + df.iloc[1][col]

        no_ratio.append(df.iloc[0][col]/col_total)
        yes_ratio.append(df.iloc[1][col]/col_total)

    n_ent = entropy(no_ratio)
    y_ent = entropy(yes_ratio)
    
    print(f'no_entropy: {n_ent}')
    print(f'yes_entropy: {y_ent}')
    print(f'weighted avg. entropy: {(n_ent*(total_no/total_val)) + (y_ent*(total_yes/total_val))}')

Write a function to compute entropy given an input of a sequence of probabilities.

In [295]:
def entropy(probs):
    h = 0
    for p in probs:
        if p != 0.0:
            h += (-p*(log2(p)))
    return h

Aggregate the tennis decision table for each value of each column. Start with Outlook below. Compute the weighted mean of the entropy for outlook (the weighted mean of the yes decision and the no decision).

In [370]:
getval(outlook)

no_entropy: 0.9709505944546686
yes_entropy: 0.9709505944546686
weighted avg. entropy: 0.9709505944546686


Compute the weighted mean of the entropy for temperature, humidity and wind as well and decide based on these values which should be the first variable chosen for a split.

In [371]:
# for humidity
getval(hum)

no_entropy: 0.8623963728982886
yes_entropy: 0.7145045417182903
weighted avg. entropy: 0.767323052854004


In [357]:
# for wind
getval(wind)

no_entropy: 1.0
yes_entropy: 0.8112781244591328
weighted avg. entropy: 0.8786787942951568


In [362]:
# for temp
getval(temp)

no_entropy: 1.5283208335737188
yes_entropy: 1.2012531249399037
weighted avg. entropy: 1.318063020880552


In [361]:
df_outlook = tennis.groupby(['Outlook','Decision']).count()

In [376]:
total = len(tennis.Decision)
rows = ['Outlook','Temp.','Humidity', 'Wind']

for row in rows:
    df = tennis.groupby([row, 'Decision']).count() 
    df = df[df.columns[0]] 
    values = df.index.get_level_values(0).unique() 
    
    probs_yes = [] 
    probs_no = [] 
    
    yes_count = 0 
    no_count = 0 
    
    yes_total = 0 
    no_total = 0 
    value_total = 0 
    
    for v in values: 
        if len(df[(v, )]) < 2: 
            probs_yes.append(1) 
            yes_total += df[v][0] 
        else: 
            no_count = df[v][0] 
            no_total += no_count 
            yes_count = df[v][1] 
            yes_total += yes_count 
            value_total = no_count + yes_count 
            
            probs_no.append(no_count/value_total) 
            probs_yes.append(yes_count/value_total) 
    
    print(f'{row}:')

    n_ent = entropy(probs_no)
    y_ent = entropy(probs_yes)
    
    print(f'no_entropy: {n_ent}')
    print(f'yes_entropy: {y_ent}')
    wae = (n_ent*(no_total/total)) + (y_ent*(yes_total/total))
    print(f'weighted avg. entropy: {wae}\n')

Outlook:
no_entropy: 0.9709505944546686
yes_entropy: 0.9709505944546686
weighted avg. entropy: 0.9709505944546686

Temp.:
no_entropy: 1.5283208335737188
yes_entropy: 1.2012531249399037
weighted avg. entropy: 1.318063020880552

Humidity:
no_entropy: 0.8623963728982886
yes_entropy: 0.7145045417182903
weighted avg. entropy: 0.767323052854004

Wind:
no_entropy: 1.0
yes_entropy: 0.8112781244591328
weighted avg. entropy: 0.8786787942951568



*Humidity is the best feature to start with for the decision tree with the lowest weighted avg. entropy.*